# Section II | Stack Visualizing!
---

# Our Question: What factors most accurately predict the score of a question on StackOverflow?

## Part 4: Training a Model to Determine Question Score


### Creating the Model






In [0]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [0]:
model_dataset_name = 'stack_overflow_post_score'

dataset = bigquery.Dataset(client.dataset(model_dataset_name))
dataset.location = 'US'
client.create_dataset(dataset)

In [0]:
%%bigquery --project $project_id

CREATE OR REPLACE MODEL `stack_overflow_post_score.question_model`
OPTIONS(model_type='logistic_reg', max_iterations=5) AS
SELECT
CASE 
WHEN q.score < 50 THEN '<50'
WHEN q.score < 100 THEN '<100'
ELSE '>=100'
END AS label,
  answer_count,
  favorite_count,
  reputation AS author_reputation,
  ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
  q.view_count,
  a.score
FROM
  `bigquery-public-data.stackoverflow.posts_questions` as q,
  `bigquery-public-data.stackoverflow.posts_answers` as a,
  `bigquery-public-data.stackoverflow.users` as users
  WHERE users.id = q.owner_user_id
  AND a.id = q.accepted_answer_id
  AND q.score <= 750
  AND EXTRACT(YEAR
  FROM
   q.creation_date) < 2018

###Training statistics for question model:

In [0]:
%%bigquery --project $project_id

# Run cell to view training stats

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stack_overflow_post_score.question_model`)


,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,4,0.033815,0.033524,122705,3.2
1,0,3,0.075518,0.075296,137429,1.6
2,0,2,0.157721,0.157606,98240,0.8
3,0,1,0.252335,0.252287,103840,0.4
4,0,0,0.323382,0.323362,55083,0.2


### b) Evaluate the question model


In [0]:
%%bigquery --project $project_id q7b

SELECT * FROM
ML.EVALUATE(MODEL `stack_overflow_post_score.question_model`,
  (
  SELECT
    CASE
      WHEN q.score < 50 THEN '<50'
      WHEN q.score < 100 THEN '<100'
      ELSE '>=100'
    END AS label,
    answer_count,
    favorite_count,
    reputation AS author_reputation,
    ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
    q.view_count,
    a.score
  FROM
    `bigquery-public-data.stackoverflow.posts_questions` AS q,
    `bigquery-public-data.stackoverflow.posts_answers` AS a,
    `bigquery-public-data.stackoverflow.users` AS users
  WHERE
    users.id = q.owner_user_id
    AND a.id = q.accepted_answer_id
    AND EXTRACT(YEAR
    FROM
      q.creation_date) = 2018
    AND EXTRACT(MONTH
    FROM
      q.creation_date) <= 6 ))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.937451,0.566959,0.999847,0.591857,1.002327,1.00117


### c) Predict question scores on questions from July and August of 2018  

In [0]:
%%bigquery --project $project_id

SELECT
    label,
    predicted_label,
  answer_count,
    favorite_count,
    author_reputation,
    q_numTags,
    view_count as question_view_count,
    score as selected_answer_score
FROM
  ML.PREDICT(MODEL `stack_overflow_post_score.question_model`, (
  SELECT
    CASE
      WHEN q.score < 50 THEN '<50'
      WHEN q.score < 100 THEN '<100'
      ELSE '>=100'
    END AS label,
    answer_count,
    favorite_count,
    reputation AS author_reputation,
    ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
    q.view_count,
    a.score
  FROM
    `bigquery-public-data.stackoverflow.posts_questions` AS q,
    `bigquery-public-data.stackoverflow.posts_answers` AS a,
    `bigquery-public-data.stackoverflow.users` AS users
  WHERE
    users.id = q.owner_user_id
    AND a.id = q.accepted_answer_id
    AND EXTRACT(YEAR
    FROM
      q.creation_date) = 2018
    AND EXTRACT(MONTH
    FROM
      q.creation_date) > 6 ))
  LIMIT 25

,label,predicted_label,answer_count,favorite_count,author_reputation,q_numTags,question_view_count,selected_answer_score
0,<50,<50,1,NaN,20,1,12,0
1,<50,<50,1,NaN,285,2,15,0
2,<50,<50,1,NaN,12,2,25,0
3,<50,<50,4,NaN,108,2,24,1
4,<50,<50,1,NaN,179,2,28,2
5,<50,<50,1,NaN,1942,3,48,0
6,<50,<50,1,NaN,1800,3,40,0
7,<50,<50,1,NaN,609,3,21,0
8,<50,<50,2,1.0,1311,3,21,1
9,<50,<50,1,NaN,2047,3,25,1


In [0]:
%%bigquery --project $project_id

CREATE OR REPLACE MODEL `stack_overflow_post_score.question_model_v2`
OPTIONS(model_type='logistic_reg', max_iterations=5) AS

WITH qa_ratio AS
 (
    SELECT
      COUNT(*)/aCount AS q2a,
      q.owner_user_id AS qa_user_id
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`q
    JOIN (
      SELECT
        owner_user_id AS userID,
        COUNT(*) AS aCount
      FROM
        `bigquery-public-data.stackoverflow.posts_answers`
      GROUP BY
        owner_user_id ) a
    ON
      a.userID = q.owner_user_id
    GROUP BY
      q.owner_user_id,
      aCount,
      q.score )
    
SELECT
 CASE 
WHEN q.score < 20 THEN '< 20'
WHEN q.score < 50 THEN '< 50'
WHEN q.score < 100 THEN '< 100'
WHEN q.score < 200 THEN '< 200'
ELSE '>=200'
END AS label,
  answer_count,
  favorite_count,
  reputation AS author_reputation,
  ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
  q.view_count as question_view_count,
  q2a as ratio_of_questions_to_answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions` AS q,
  `bigquery-public-data.stackoverflow.users` AS users,
  qa_ratio
WHERE
  users.id = q.owner_user_id
  AND qa_ratio.qa_user_id = users.id
  AND q.score > 10
  AND EXTRACT(YEAR
  FROM
    q.creation_date) < 2018

### Training Statistics for V2 of the Question Model:

In [0]:
%%bigquery --project $project_id

# Run cell to view training stats

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stack_overflow_post_score.question_model_v2`)


,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,4,0.217334,0.217912,102485,3.2
1,0,3,0.241879,0.242203,161236,1.6
2,0,2,0.272556,0.272713,139609,0.8
3,0,1,0.297357,0.297440,184641,0.4
4,0,0,0.313043,0.313082,65255,0.2


### Evaluate on V2 of the Question Model:


In [0]:
%%bigquery --project $project_id q7b

SELECT * FROM
ML.EVALUATE(MODEL `stack_overflow_post_score.question_model_v2`,
  (
  WITH qa_ratio AS
 (
    SELECT
      COUNT(*)/aCount AS q2a,
      q.owner_user_id AS qa_user_id
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`q
    JOIN (
      SELECT
        owner_user_id AS userID,
        COUNT(*) AS aCount
      FROM
        `bigquery-public-data.stackoverflow.posts_answers`
      GROUP BY
        owner_user_id ) a
    ON
      a.userID = q.owner_user_id
    GROUP BY
      q.owner_user_id,
      aCount,
      q.score )
    
SELECT
 CASE 
WHEN q.score < 20 THEN '< 20'
WHEN q.score < 50 THEN '< 50'
WHEN q.score < 100 THEN '< 100'
WHEN q.score < 200 THEN '< 200'
ELSE '>=200'
END AS label,
  answer_count,
  favorite_count,
  reputation AS author_reputation,
  ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
  q.view_count as question_view_count,
  q2a as ratio_of_questions_to_answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions` AS q,
  `bigquery-public-data.stackoverflow.users` AS users,
  qa_ratio
WHERE
  users.id = q.owner_user_id
  AND qa_ratio.qa_user_id = users.id
  AND q.score > 10
    AND EXTRACT(YEAR
    FROM
      q.creation_date) = 2018
    AND EXTRACT(MONTH
    FROM
      q.creation_date) <= 6 ))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,NaN,0.223959,0.761061,0.205719,1.573562,0.753277


There is still an uneven distribution of values across these scores. While there are approximately 250k questions with scores between 20 and 50 points, there are an order of magnitude less between 100 and 200 and between 200 and the max. We hypothesize that this is why our precision was not a number.  In our next experiment we try to better balance the number of entries between each of the categories. We still exclude all questions with scores less than 10 and use the buckets s < 20, s < 50 and s > 50. The first two buckets have over 200k entries each that fit those categories and the second bucket has just under 70k. We felt these sizes were comparable enough to retry on our model.

In [0]:
%%bigquery --project $project_id

CREATE OR REPLACE MODEL `stack_overflow_post_score.question_model_v3`
OPTIONS(model_type='logistic_reg', max_iterations=5) AS

WITH qa_ratio AS
 (
    SELECT
      COUNT(*)/aCount AS q2a,
      q.owner_user_id AS qa_user_id
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`q
    JOIN (
      SELECT
        owner_user_id AS userID,
        COUNT(*) AS aCount
      FROM
        `bigquery-public-data.stackoverflow.posts_answers`
      GROUP BY
        owner_user_id ) a
    ON
      a.userID = q.owner_user_id
    GROUP BY
      q.owner_user_id,
      aCount,
      q.score )
    
SELECT
CASE
 WHEN q.score < 20 THEN '< 20'
WHEN q.score < 50 THEN '< 50'
ELSE '>=50'
END AS label,
  answer_count,
  a.score,
  favorite_count,
  reputation AS author_reputation,
  ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
  q.view_count as question_view_count,
  q2a as ratio_of_questions_to_answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions` AS q,
  `bigquery-public-data.stackoverflow.posts_answers` AS a,
  `bigquery-public-data.stackoverflow.users` AS users,
  qa_ratio
WHERE
  users.id = q.owner_user_id
  AND qa_ratio.qa_user_id = users.id
  AND a.id = q.accepted_answer_id
  AND q.score > 10
  AND EXTRACT(YEAR
  FROM
    q.creation_date) < 2018

Executing query with job ID: 93420c87-4dcc-4299-9d36-83c35e1edabf
Query executing: 740.44s

ValueError: ignored

In [0]:
%%bigquery --project $project_id

# Run cell to view training stats

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stack_overflow_post_score.question_model_v3`)


,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,4,0.275468,0.274151,88549,3.2
1,0,3,0.293709,0.292662,80975,1.6
2,0,2,0.316116,0.315464,76510,0.8
3,0,1,0.337869,0.337559,78467,0.4
4,0,0,0.354750,0.354687,50147,0.2


In [0]:
%%bigquery --project $project_id q7b

SELECT * FROM
ML.EVALUATE(MODEL `stack_overflow_post_score.question_model_v3`,
  (
  WITH qa_ratio AS
 (
    SELECT
      COUNT(*)/aCount AS q2a,
      q.owner_user_id AS qa_user_id
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`q
    JOIN (
      SELECT
        owner_user_id AS userID,
        COUNT(*) AS aCount
      FROM
        `bigquery-public-data.stackoverflow.posts_answers`
      GROUP BY
        owner_user_id ) a
    ON
      a.userID = q.owner_user_id
    GROUP BY
      q.owner_user_id,
      aCount,
      q.score )
    
SELECT
CASE
 WHEN q.score < 20 THEN '< 20'
WHEN q.score < 50 THEN '< 50'
ELSE '>=50'
END AS label,
  answer_count,
  a.score,
  favorite_count,
  reputation AS author_reputation,
  ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
  q.view_count as question_view_count,
  q2a as ratio_of_questions_to_answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions` AS q,
  `bigquery-public-data.stackoverflow.posts_answers` AS a,
  `bigquery-public-data.stackoverflow.users` AS users,
  qa_ratio
WHERE
  users.id = q.owner_user_id
  AND qa_ratio.qa_user_id = users.id
  AND a.id = q.accepted_answer_id
  AND q.score > 10
    AND EXTRACT(YEAR
    FROM
      q.creation_date) = 2018
    AND EXTRACT(MONTH
    FROM
      q.creation_date) <= 6 ))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.616093,0.398623,0.748853,0.396464,1.057003,0.717084


In [0]:
%%bigquery --project $project_id

SELECT
    label,
    predicted_label,
  answer_count,
  score as answer_score,
    favorite_count,
    author_reputation,
    q_numTags,
    ratio_of_questions_to_answers
FROM
  ML.PREDICT(MODEL `stack_overflow_post_score.question_model_v3`, (
   WITH qa_ratio AS
 (
    SELECT
      COUNT(*)/aCount AS q2a,
      q.owner_user_id AS qa_user_id
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`q
    JOIN (
      SELECT
        owner_user_id AS userID,
        COUNT(*) AS aCount
      FROM
        `bigquery-public-data.stackoverflow.posts_answers`
      GROUP BY
        owner_user_id ) a
    ON
      a.userID = q.owner_user_id
    GROUP BY
      q.owner_user_id,
      aCount,
      q.score )
    
SELECT
CASE
 WHEN q.score < 20 THEN '< 20'
WHEN q.score < 50 THEN '< 50'
ELSE '>=50'
END AS label,
  answer_count,
  a.score,
  favorite_count,
  reputation AS author_reputation,
  ARRAY_LENGTH(SPLIT(q.tags, "|")) AS q_numTags,
  q.view_count as question_view_count,
  q2a as ratio_of_questions_to_answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions` AS q,
  `bigquery-public-data.stackoverflow.posts_answers` AS a,
  `bigquery-public-data.stackoverflow.users` AS users,
  qa_ratio
WHERE
  users.id = q.owner_user_id
  AND qa_ratio.qa_user_id = users.id
  AND a.id = q.accepted_answer_id
  AND q.score > 10
    AND EXTRACT(YEAR
    FROM
      q.creation_date) = 2018
    AND EXTRACT(MONTH
    FROM
      q.creation_date) > 6 ))
  LIMIT 25

,label,predicted_label,answer_count,answer_score,favorite_count,author_reputation,q_numTags,ratio_of_questions_to_answers
0,< 20,< 20,5,16,0,18666,3,0.006579
1,< 20,< 20,6,10,1,451,2,0.200000
2,< 20,< 20,3,1,1,143,5,0.500000
3,< 20,< 20,1,14,1,1713,3,50.000000
4,< 20,< 20,1,16,1,41881,4,0.000968
5,< 20,< 20,2,9,2,13742,4,0.002033
6,< 20,< 20,3,23,2,745,2,0.666667
7,< 20,< 20,5,29,2,1900,2,5.533333
8,< 20,< 20,1,8,3,3943,5,0.018519
9,< 20,< 20,5,8,3,1536,1,0.016949


Having found the appropriate bucket sizes, we additionally improved upon this model's accuracy by reintroducing the selected answer score.